In [ ]:
import pandas as pd

In [ ]:
# Import the process data in a dataframe as example from excel file
# you need the position of your process step, your activity name and the position of the previous process step
df = pd.read_excel('T_CONFO.xlsx')

# PREPROCESSING THE DATA FRAME
# create label list for each SERG
# Fill your dataframe etc... do what ever you need to do with it
# for example in DF the columns names in the following code are used for:
# SERG --> activity name
# RF --> Position of the Activity in Process
# VG --> Previous Activitys cut by ';'
df['VG'].fillna('-1;', inplace=True)
df.columns = {'RF':'RF','SERG':'EVENT','VG':'VG','KUERZEL':'KÜRZEL'}


# Fill the labels list with the activity names
labels = df['SERG'].to_list()

##  CREATE TRANSITION NUMBERS AND MATCH WITH LABEL ##
a = 0
transitions = []
t_label = []
for label in labels:
    a += 1
    transitions.append('T_' + str(a))
    t_label.append([label, 'T_' + str(a)])

## CREATE CONNECTIONS ##
# creating Connection between all Activities
keys = t_label
values = []
a = 0
for label,key in keys:
    rfolge = int(df['RF'][df.SERG == label])
    follows = df['VG'][df.SERG == label].str.split(';')
    for follow in follows:
        follow = ' '.join(follow).split()
        for fo in follow:
            for label2,key2 in keys:
                vg = int(df['RF'][df.SERG == label2])
                if int(fo) == vg:
                    a += 1
                    values.append([key , key2 , 'P_' + str(a)])
                    break
                if rfolge == df2['RF'].min():
                    a += 1
                    values.append([key , 'NONE', 'P_' + str(a)])
                    break
                    
## CREATE THE EDGES AND PLACES ##
arcs =[]
places = []
for c,b,place in values:
    if b != 'NONE': arcs.append([b,place])
    arcs.append([place,c])
    places.append(place)

arcs.append([values [len(values) -1][0],'P_'+str(a+1)])
places.append('P_'+str(a+1))

## DEFINE START AND END PLACE ##
s_list = [places[0], places[len(places)-1]]

## TRANSFORM THE DATA IN THE LISTS FOR CONFORMANCE ##
newlabel = '['
for t in t_label:
    label = "['" + "' \"".join(t) + '"]'
    newlabel = newlabel + label
newlabel = newlabel + ']'
place = '["'+ '" "'.join(places) + '"]'
transition = '["'+ '" "'.join(transitions) + '"]'
s_e = '["'+ '"], ["'.join(s_list) + '"]'
newarc = '['
for arclist in arcs:
    arc = '["'+ '" "'.join(arclist) + '"]'
    newarc = newarc + arc
newarc = newarc + ']'

## CONCATENATE THE LISTS INTO STRING WITH CONFORMANCE ##
confo = 'CONFORMANCE("Your Activity Column",' + place + ', ' + transition + ', ' + newarc + ',' + newlabel + ',' + s_e + ')'

## PRINT RESULT
print(confo)